In [42]:
import numpy as np

In [43]:
# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))

def sigmoid(x):
    if isinstance(x, list):
        sigmoids = []
        for value in x:
            sigmoids.append(sigmoid(value))
        return sigmoids
    else:
        return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    if isinstance(x, list):
        sigmoids_derivative = []
        for value in x:
            sigmoids_derivative.append(sigmoid_derivative(value))
        return sigmoids_derivative
    else:
        s = sigmoid(x)
        return s * (1 - s)

In [44]:
# example in the lecture 4

X = np.array([0, 0, 1])

Weights = [
    np.array([
        [0.21, 0.15, -0.3],
        [0.10, 0.25, -0.4]
    ]),
    np.array([
        [-0.2, 0.3, -0.4]
    ])
]


In [45]:
import numpy as np

# Input vector
X = np.array([0, 0, 1])  # Include bias as the last input

# Weights for the layers
Weights = [
    np.array([
        [0.21, 0.15, -0.3],
        [0.10, -0.4, 0.25]
    ]),
    np.array([
        [-0.2, 0.3, -0.4]
    ])
]
activations = []
sigmasList = []
# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Feedforward function
def feedForward(X):
    global activations, nets
    activations = [X] 
    nets = []
    for i in range(len(Weights)):
        # Compute the input for the current layer
        layer_input = np.dot(activations[i], Weights[i].T)
        nets.append(layer_input)
        # print("layer_input", layer_input)
        
        # Apply activation function (sigmoid for hidden layers, no activation for output layer)
        layer_input = sigmoid(layer_input)
        if i != len(Weights) - 1:  # If not the output layer
            # Add bias to the next layer input
            layer_input = np.append(layer_input, 1)
        
        # Append the result to the activations list
        activations.append(layer_input)
        # print("activations", activations)
        # print("nets = ", nets)
        # print("nest shape : ", np.array(nets).shape)
    
    return activations[-1]  # Return the output of the final layer


print(feedForward(X))
# print("nets = ", nets)
print("activation" ,activations)
def backPropagation(X, y, learning_rate = 0.1):
    global sigmasList

    # Feedforward
    output_activation = feedForward(X)

    sigmasList = []
    sigmaOutput = (y - output_activation) * output_activation * (1 - output_activation)
    print("sigmaOutput", sigmaOutput)

    sigmas = sigmaOutput
    for i in range(len(Weights)-1, 0, -1):
        sigmasList.append(sigmas * Weights[i] * sigmoid_derivative(nets[i]))

    sigmasList.reverse()
    sigmasList.append(sigmaOutput)


backPropagation(X, 0, learning_rate=0.1)
print("Sigma" , sigmasList)
# under creation method
def ModifyWeights(learningRate):
    for i, sigmas in enumerate(sigmasList):
        Weights[i] = Weights[i] + learningRate * sigmas * activations[i]

print("Weights before" , Weights)      
ModifyWeights(0.1)
print("Weights after" , Weights)


[0.42153907]
activation [array([0, 0, 1]), array([0.42555748, 0.5621765 , 1.        ]), array([0.42153907])]
sigmaOutput [-0.10278972]
Sigma [array([[ 0.00501293, -0.00751939,  0.01002586]]), array([-0.10278972])]
Weights before [array([[ 0.21,  0.15, -0.3 ],
       [ 0.1 , -0.4 ,  0.25]]), array([[-0.2,  0.3, -0.4]])]
Weights after [array([[ 0.21      ,  0.15      , -0.29899741],
       [ 0.1       , -0.4       ,  0.25100259]]), array([[-0.20437429,  0.2942214 , -0.41027897]])]
